In [1]:
import matplotlib.pyplot as plt
import numpy as np

from numpy import sin, cos, pi, arcsin, inf
%matplotlib inline

In [84]:
def thetaCritical(m, n_list):
    """
    :param m: layer containing the emitting atom
    :param n_list: list of refractive indices of the layers
    :return: Return the angle at which TIRF occurs between the layer containing the atom and the cladding with
    the largest refractive index, or pi/2, whichever comes first.
    """

    # Evaluate largest refractive index of claddings
    n_clad = max(n_list[0], n_list[-1])

    # Using Snell's law evaluate the critical angle or return pi/2 if does not exist
    if n_clad/n_list[m] < 1:
        return arcsin(n_clad/n_list[m])
    else:
        return pi/2

In [2]:
d_list = [inf, 2000, 1000, inf]
# list of refractive indices
n_list = [1.5, 1.5, 3, 3]
x_step=1

In [3]:
# Set first and last layer thicknesses to zero (from inf) - helps for summing later in program
d_list[0] = 0
d_list[-1] = 0

In [5]:
# Start position of each layer
d_cumsum = np.cumsum(d_list)
# x positions to evaluate E field at
x_pos = np.arange((x_step / 2.0), sum(d_list), x_step)

# get x_mat (specifies what layer number the corresponding point in x_pos is in):
comp1 = np.kron(np.ones((num_layers, 1)), x_pos)
comp2 = np.transpose(np.kron(np.ones((len(x_pos), 1)), d_cumsum))

x_mat = sum(comp1 > comp2, 0)

In [43]:
x_mat

array([1, 1, 1, ..., 2, 2, 2])

In [44]:
layer = 2

In [48]:
np.eye(2)

array([[ 1.,  0.],
       [ 0.,  1.]])

In [63]:
a = [1, 2]*4

In [65]:
a.append(5)
a

[1, 2, 1, 2, 1, 2, 1, 2, 5, 5]

In [ ]:
import scipy as sp
import numpy as np

from numpy import pi, exp, sin, inf, sqrt
from scipy.interpolate import interp1d


def I_mat(nj, nk, n0, pol, th_0):
    """
    Calculates the interference matrix between two layers.
    :param nj: First layer refractive index
    :param nk: Second layer refractive index
    :param n0: Refractive index of incident transparent medium
    :param pol: Polarisation of incoming light ('s' or 'p')
    :param th_0: Angle of incidence of light (0 for normal, pi/2 for glancing)
    :return: I-matrix
    """
    # transfer matrix at an interface
    qj = sqrt(nj**2 - n0.real**2 * sin(th_0)**2)
    qk = sqrt(nk**2 - n0.real**2 * sin(th_0)**2)

    if pol == 's':
        r = (qj - qk) / (qj + qk)
        t = (2 * qj) / (qj + qk)

    elif pol == 'p':
        r = (qj * nk**2 - qk * nj**2) / (qj * nk**2 + qk * nj**2)
        t = (2 * nj * nk * qj) / (qj * nk**2 + qk * nj**2)

    else:
        raise ValueError("Polarisation must be 's' or 'p' when angle of incidence is"
                         " not 90$\degree$s")

    if t == 0:
        raise ValueError('Transmission is zero, cannot evaluate I+mat. Check input parameters.')

    return (1/t) * np.array([[1, r], [r, 1]], dtype=complex)


def L_mat(nj, dj, lam_vac, n0, th_0):
    """
    Calculates the propagation.
    :param n: complex dielectric constant
    :param d: thickness
    :param lam_vac: wavelength
    :return:  L-matrix
    """
    qj = sp.sqrt(nj**2 - n0.real**2 * np.sin(th_0)**2)
    eps = (2*pi*qj) / lam_vac
    return np.array([[exp(-1j*eps*dj), 0], [0, exp(1j*eps*dj)]], dtype=complex)


def TransferMatrix(d_list, n_list, lam_vac, th_0, pol, x_step=1, glass=False):
    """
    Evaluate the transfer matrix over the entire structure.
    :param pol: polarisation of incoming light ('s' or 'p')
    :param n_list: list of refractive indices for each layer (can be complex)
    :param d_list: list of thicknesses for each layer
    :param th_0: angle of incidence (0 for normal, pi/2 for glancing)
    :param lam_vac: vacuum wavelength of light
    :param glass: Bool. If there is a thick superstrate present then make true as interference
                        in this layer is negligible
    :return: Dictionary of all input and output params related to structure
    """
    # convert lists to numpy arrays if they're not already.
    n_list = np.array(n_list, dtype=complex)
    d_list = np.array(d_list, dtype=float)

    # input tests
    if ((hasattr(lam_vac, 'size') and lam_vac.size > 1) or (hasattr(th_0, 'size')
                                                            and th_0.size > 1)):
        raise ValueError('This function is not vectorized; you need to run one '
                         'calculation at a time (1 wavelength, 1 angle, etc.)')
    if (n_list.ndim != 1) or (d_list.ndim != 1) or (n_list.size != d_list.size):
        raise ValueError("Problem with n_list or d_list!")
    if (d_list[0] != inf) or (d_list[-1] != inf):
        raise ValueError('d_list must start and end with inf!')
    if type(x_step) != int:
        raise ValueError('x_step must be an integer otherwise. Reduce SI unit'
                         'inputs for thicknesses and wavelengths for greater resolution ')
    if th_0 >= pi/2 or th_0 <= -pi/2:
        raise ValueError('The light is not incident on the structure. Check input theta '
                         '(0 <= theta < pi/2')

    # Set first and last layer thicknesses to zero (from inf) - helps for summing later in program
    d_list[0] = 0
    d_list[-1] = 0

    num_layers = np.size(d_list)
    n = n_list
    # Start position of each layer
    d_cumsum = np.cumsum(d_list)
    # x positions to evaluate E field at
    x_pos = np.arange((x_step / 2.0), sum(d_list), x_step)

    # get x_mat - specifies what layer the corresponding point in x_pos is in
    comp1 = np.kron(np.ones((num_layers, 1)), x_pos)
    comp2 = np.transpose(np.kron(np.ones((len(x_pos), 1)), d_cumsum))
    x_mat = sum(comp1 > comp2, 0)

    # calculate the total system transfer matrix S
    # S = I_mat(n[0], n[1], n[0], pol, th_0)
    # for layer in range(1, num_layers - 1):
    #     mI = I_mat(n[layer], n[layer + 1], n[0], pol, th_0)
    #     mL = L_mat(n[layer], d_list[layer], lam_vac, n[0], th_0)
    #     S = S @ mI @ mL

    S = np.eye(2)
    for layer in range(1, num_layers - 2):
        mI = I_mat(n[layer-1], n[layer], n[0], pol, th_0)
        mL = L_mat(n[layer], d_list[layer], lam_vac, n[0], th_0)
        S = S @ mI @ mL
    S = S @ I_mat(n[num_layers-2], n[num_layers-1], n[0], pol, th_0)

    # JAP Vol 86 p.487 Eq 9 and 10: Power Reflection and Transmission
    R = abs(S[1, 0] / S[0, 0]) ** 2
    T = abs(1 / S[0, 0]) ** 2  # note this is incorrect https://en.wikipedia.org/wiki/Fresnel_equations

    # calculate primed transfer matrices for info on field inside the structure
    E = np.zeros(len(x_pos), dtype=complex)  # Initialise E field
    E_avg = np.zeros(num_layers)
    for layer in range(1, num_layers):
        qj = sp.sqrt(n[layer]**2 - n[0].real**2 * np.sin(th_0)**2)
        eps = (2 * np.pi * qj) / lam_vac
        dj = d_list[layer]
        x_indices = np.where(x_mat == layer)
        # Calculate depth into layer
        x = x_pos[x_indices] - d_cumsum[layer - 1]
        # Calculate S_Prime
        S_prime = np.eye(2)
        for v in range(1, layer-1):
            mI = I_mat(n[v - 1], n[v], n[0], pol, th_0)
            mL = L_mat(n[v], d_list[v], lam_vac, n[0], th_0)
            S_prime = S_prime @ mL @ mI
        S_prime = S_prime @ I_mat(n[layer-1], n[layer], n[0], pol, th_0)

        # Calculate S_dprime (double prime)
        S_dprime = np.eye(2)
        for v in range(layer + 1, num_layers - 2):
            mI = I_mat(n[v-1], n[v], n[0], pol, th_0)
            mL = L_mat(n[v], d_list[v + 1], lam_vac, n[0], th_0)
            S_dprime = S_dprime @ mI @ mL
        S_dprime = S_dprime @ I_mat(n[num_layers-2], n[num_layers-1], n[0], pol, th_0)

        # Electric Field Profile
        num = S_dprime[0, 0] * exp(-1j*eps*(dj-x)) + S_dprime[1, 0] * exp(1j*eps*(dj-x))
        den = S_prime[0, 0] * S_dprime[0, 0] * exp(-1j*eps*dj) + S_prime[0, 1] * S_dprime[1, 0] * exp(1j*eps*dj)
        E[x_indices] = num / den

        # Average E field inside the layer
        E_avg[layer] = sum(abs(E[x_indices])**2) / (x_step*d_list[layer])

    # |E|^2
    E_square = abs(E[:]) ** 2

    # Absorption coefficient in 1/cm
    absorption = np.zeros(num_layers)
    for layer in range(1, num_layers):
        absorption[layer] = (4 * np.pi * np.imag(n[layer])) / (lam_vac * 1.0e-7)

    return {'E_square': E_square, 'absorption': absorption, 'x_pos': x_pos,  # output functions of position
            'R': R, 'T': T, 'E': E, 'E_avg': E_avg,  # output overall properties of structure
            'd_list': d_list, 'th_0': th_0, 'n_list': n_list, 'lam_vac': lam_vac, 'pol': pol,  # input structure
            }


class LifetimeTmm:
    def __init__(self, d_list, n_list, x_step=1):
        """
        Initilise with the structure of the material to be simulated
        """
        self.d_list = d_list
        self.n_list = n_list

        # TODO problem if one of the thicknesses in d_list is inf
        self.x_pos = np.arange((x_step / 2.0), sum(d_list), x_step)

    def __call__(self, lam_vac, th_0, pol, x_step=1):
        """
        Call the simulation for the specific structure with the wavelength(s) to be simulated and (optionally)
        the angle of incidence, polarization and resolution in x
        """
        return TransferMatrix(self.d_list, self.n_list, lam_vac, th_0, pol, x_step)


